In [6]:
import json
import os
import numpy as np
# import torch
from collections import OrderedDict
from copy import deepcopy
import time
from droid.robot_env import RobotEnv

In [7]:
action_space = "cartesian_position"
gripper_action_space = "position"


imsize = 256

camera_kwargs = dict(
    hand_camera=dict(image=False, concatenate_images=False, resolution=(imsize, imsize), resize_func=None),
    varied_camera=dict(image=False, concatenate_images=False, resolution=(imsize, imsize), resize_func=None),
)

env = RobotEnv(
    action_space=action_space,
    gripper_action_space=gripper_action_space,
    camera_kwargs=camera_kwargs
)

In [8]:
env.reset(randomize=False)

In [69]:
top_pose = [0.4350,
0.0,
0.75,
-3.135387735082638,
0.0117385168564208,
-0.008736370437155985]
# bottom_pose = [0.4321105480194092,
#  0,
#  0.150000000000000,
#  -3.141084945536129,
#  0.022017310521937228,
#  -0.01628570321540242]
bottom_pose = [0.4350,
0.0,
0.19,
-3.135387735082638,
0.0117385168564208,
-0.008736370437155985]
home_pose = [0.4350,
0.0,
0.4,
-3.135387735082638,
0.0117385168564208,
-0.008736370437155985]

In [43]:
def run(env, pose, duration=1.0, grp=False):
    pose = np.array(pose)
    pose = np.concatenate([pose, np.array([1.0]) if grp else np.array([0.0])])
    for i in range(int(duration * 15)):
        env.step(pose)
        time.sleep(1/15)

In [82]:
def dry_run(env, pose, grp=False):
    pose = np.array(pose)
    pose = np.concatenate([pose, np.array([1.0]) if grp else np.array([0.0])])
    env.step(pose)
    time.sleep(1/30)

In [78]:
new_pose_bottom = bottom_pose
run(env, bottom_pose, duration=2.0, grp=False)

In [79]:
run(env, bottom_pose, duration=2.0, grp=True)

In [68]:
run(env, home_pose, duration=2.0, grp=True)

In [56]:
run(env, home_pose, duration=2.0, grp=False)

In [ ]:
resolution = 70
radius = 0.10

circle_start_pose = [0.4350 - radius,
0.0,
0.4,
-3.135387735082638,
0.0117385168564208,
-0.008736370437155985]

run(env, circle_start_pose, duration=2.0, grp=True)

for i in range(resolution):
    if i < resolution / 2:
        circle_current_pose = [0.4350 - radius * np.cos(2*np.pi / resolution * i),
            0.0,
            0.4 - radius * np.sin(2*np.pi / resolution * i),
            -3.135387735082638,
            0.0117385168564208 + np.pi/6 - (np.pi/2.5+np.pi/6) * i / (resolution / 2),
            -0.008736370437155985
        ]
        dry_run(env, circle_current_pose, grp=True)
    elif i < resolution * 3 / 4 :
        circle_current_pose = [0.4350 - radius * np.cos(2*np.pi / resolution * i),
            0.0,
            0.4 - radius * np.sin(2*np.pi / resolution * i),
            -3.135387735082638,
            0.0117385168564208 - (np.pi/2.5),
            -0.008736370437155985
        ]
        dry_run(env, circle_current_pose, grp=True)
    else:
        circle_current_pose = [0.4350 - radius * np.cos(2*np.pi / resolution * i),
            0.0,
            0.4 - radius * np.sin(2*np.pi / resolution * i),
            -3.135387735082638,
            0.0117385168564208 - (np.pi/2.5) * (resolution - i) / (resolution / 4),
            -0.008736370437155985
        ]
        dry_run(env, circle_current_pose, grp=False)

run(env, circle_start_pose, grp=False)


In [108]:
for j in range(10):
    if j % 2 == 0:
        shake_pose = [0.4350+0.1,
        0.0+0.1,
        0.6,
        -3.135387735082638 + 0.3,
        0.0117385168564208 - 0.3,
        -0.008736370437155985]
    else:
        shake_pose = [0.4350-0.1,
        0.0-0.1,
        0.6,
        -3.135387735082638 - 0.3,
        0.0117385168564208 + 0.3,
        -0.008736370437155985]

    run(env, shake_pose, duration=0.5, grp=True)

In [11]:
obs_dict = env.get_observation()
obs_dict.keys()

dict_keys(['timestamp', 'robot_state', 'image', 'camera_type', 'camera_extrinsics', 'camera_intrinsics'])

In [41]:
type(obs_dict["image"])
obs_dict["image"]["14013996_left"]
print(obs_dict["image"]["14013996_left"].shape)
img = obs_dict["image"]["14013996_left"][..., :3]
from PIL import Image
img_pil = Image.fromarray(img)
img_pil.show()

(720, 1280, 4)


In [ ]:
def environment_reset(env):
    
    ## somehow decide 

In [129]:
run(env, top_pose, duration=2.0, grp=False)
run(env, bottom_pose, duration=2.0, grp=False)
run(env, bottom_pose, duration=1.0, grp=True)
run(env, top_pose, duration=2.0, grp=True)

x_offset = 0.1
new_top_pose = top_pose + np.array([x_offset, 0, -0.1, 0, 0, 0])
new_bottom_pose = bottom_pose + np.array([x_offset, 0, 0, 0, 0, 0])
run(env, new_top_pose, duration=1.0, grp=True)
run(env, new_bottom_pose, duration=1.0, grp=True)

In [131]:
env.reset()